# ATGU Workshop -- Common Variant Analysis

In this section, we are learning how to 

1) Use simple python code  and Jupyter notebook

2) Setting Hail up in your environment

3) Simple descriptive analysis with Hail

4) Common variant analysis with Hail

# Hail on Jupyter

Part of what we think is so exciting about Hail is that it has coincided with a larger shift in the data science community.

Three years ago, most computational biologists at Broad analyzed genetic data using command-line tools, and took advantage of research compute clusters by explicitly using scheduling frameworks like LSF or Sun Grid Engine.

Now, they have the option to use Hail in interactive Python notebooks backed by thousands of cores on public compute clouds like [Google Cloud](https://cloud.google.com/), [Amazon Web Services](https://aws.amazon.com/), or [Microsoft Azure](https://azure.microsoft.com/).

# Using Jupyter
### Running cells
Evaluate cells using SHIFT + ENTER. Select the next cell and run it

In [1]:
print('Hello, world')

Hello, world


### Modes

Jupyter has two modes, a **navigation mode** and an **editor mode**.

#### Navigation mode:

 - <font color="blue"><strong>BLUE</strong></font> cell borders
 - `UP` / `DOWN` move between cells
 - `ENTER` while a cell is selected will move to **editing mode**.
 - Many letters are keyboard shortcuts! This is a common trap.
 
#### Editor mode:

 - <font color="green"><strong>GREEN</strong></font> cell borders
 - `UP` / `DOWN`/ move within cells before moving between cells.
 - `ESC` will return to **navigation mode**.
 - `SHIFT + ENTER` will evaluate a cell and return to **navigation mode**.

### Cell types

There are several types of cells in Jupyter notebooks. The two you will see here are **Markdown** (text) and **Code**.

In [2]:
# This is a code cell
my_variable = 5

**This is a markdown cell**, so even if something looks like code (as below), it won't get executed!

my_variable += 1

### Tips and tricks

Keyboard shortcuts:

 - `SHIFT + ENTER` to evaluate a cell
 - `ESC` to return to navigation mode
 - `y` to turn a markdown cell into code
 - `m` to turn a code cell into markdown
 - `a` to add a new cell **above** the currently selected cell
 - `b` to add a new cell **below** the currently selected cell
 - `d, d` (repeated) to delete the currently selected cell
 - `TAB` to activate code completion
 
To try this out, create a new cell below this one using `b`, and print `my_variable` by starting with `print(my` and pressing `TAB`!

# Set up our Python environment

In addition to Hail, we import a few methods from the Hail plotting library. We'll see examples soon!
For further installation instructions, please visit our Getting Started page (https://hail.is/docs/0.2/getting_started.html)

In [3]:
import hail as hl

The demonstration materials are designed to work on a small (~20MB) downsampled chunk of the public 1000 Genomes dataset.


It is possible to call command-line utilities from Jupyter by prefixing a line with a !:

In [4]:
! ls -1 resources/

1kg.fam*
1kg.mt/
1kg.vcf.bgz*
1kg_annotations.txt*
pca_scores.ht/
post_qc.mt/
sample_qc.ht/
true_pops.txt*


# Explore genetic data with Hail

#### Learning Objectives:

- To be comfortable exploring Hail data structures.
- To understand categories of functionality for performing QC.

### Import data from VCF

The [Variant Call Format (VCF)](https://en.wikipedia.org/wiki/Variant_Call_Format) is a common file format for representing genetic data collected on multiple individuals (samples).

Hail's [import_vcf](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_vcf) function can read this format.

However, VCF is a text format that is easy for humans to read, but very inefficient to process on a computer. 

The first thing we do is import (`import_vcf`) and convert the `VCF` file into a Hail native file format. This is done by using the `write` method below. The resulting file is **much** faster to process because it is scalable and easily parallelizable.

Let's read in a chunk of data, specifically from 1000 Genomes

In [5]:
YOUR_BUCKET = ...

In [6]:
hl.import_vcf('resources/1kg.vcf.bgz', min_partitions=4).write('resources/1kg.mt', overwrite=True)

Initializing Hail with default parameters...
SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/Users/pschulz/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://10.0.0.135:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.121-7fb600840651
LOGGING: writing to /Users/pschulz/Downloads/ATGU-Hail_Workshop2020-master/hail-20230911-14

### Read 1KG into Hail

We represent genetic data as a Hail [`MatrixTable`](https://hail.is/docs/0.2/overview/matrix_table.html), and name our variable `mt` to indicate this.

In [7]:
mt = hl.read_matrix_table('resources/1kg.mt')

### What is a `MatrixTable`?

Let's explore it!

You can see:
 - **numeric** types:
     - integers (`int32`, `int64`), e.g. `5`
     - floating point numbers (`float32`, `float64`), e.g. `5.5` or `3e-8`
 - **strings** (`str`), e.g. `"Foo"`
 - **boolean** values  (`bool`) e.g. `True`
 - **collections**:
     - arrays (`array`), e.g. `[1,1,2,3]`
     - sets (`set`), e.g. `{1,3}`
     - dictionaries (`dict`), e.g. `{'Foo': 5, 'Bar': 10}`
 - **genetic data types**:
     - loci (`locus`), e.g. `[GRCh37] 1:10000` or `[GRCh38] chr1:10024`
     - genotype calls (`call`), e.g. `0/2` or `1|0`

In [8]:
mt.describe(widget=True)

### `show`

You can show individual fields like the sample ID (`s`), 

In [9]:
mt.s.show()

""
s
str
"""HG00096"""
"""HG00099"""
"""HG00105"""
"""HG00118"""
"""HG00129"""
"""HG00148"""
"""HG00177"""
"""HG00182"""


the locus (`locus`)

In [10]:
mt.locus.show()

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1509414,"[""AG"",""A""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"
1:2044130,"[""GTT"",""G""]"
1:2169908,"[""G"",""T""]"


or the called genotype (`GT`):

In [11]:
mt.GT.show()

,,,,,
,,'HG00096','HG00099','HG00105','HG00118'
locus,alleles,GT,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call,call
1:904165,"[""G"",""A""]",0/0,0/0,0/0,0/0
1:909917,"[""G"",""A""]",0/0,0/0,0/0,0/0
1:986963,"[""C"",""T""]",0/0,0/0,0/0,0/0
1:1509414,"[""AG"",""A""]",0/0,0/0,0/0,0/0
1:1563691,"[""T"",""G""]",NA,0/0,0/0,0/0
1:1707740,"[""T"",""G""]",0/1,0/1,0/1,0/0
1:2044130,"[""GTT"",""G""]",0/1,0/1,0/1,0/1


### `summarize`
`summarize` Prints (potentially) useful information about any field or object:

In [12]:
mt.DP.summarize()

Non-missing,4399949 (98.43%)
Missing,70370 (1.57%)
Minimum,0
Maximum,150
Mean,7.26
Std Dev,4.40


In [13]:
mt.AD.summarize()

Non-missing,4399949 (98.43%)
Missing,70370 (1.57%)
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,8799898 (100.00%)
Missing,0
Minimum,0
Maximum,150
Mean,3.63
Std Dev,4.34


### `count`

`MatrixTable.count` returns a tuple with the number of rows (variants) and number of columns (samples).

In [14]:
mt.count()

(13033, 343)

### Hail has functions built for genetics

For example, `hl.summarize_variants` prints useful statistics about the genetic variants in the dataset. These are not part of the generic `summarize()` function, which must support all kinds of data, not just variant data!

In [15]:
hl.summarize_variants(mt)

Number of alleles,Count
2,13033
Allele type,Count
SNP,12800
Deletion,138
Insertion,95
Metric,Value
Transitions,9840
Transversions,2960
Ratio,3.32
Contig,Count


# Annotation and quality control

## Integrate sample information

This is a text file containing phenotype information:

In [16]:
! head resources/1kg_annotations.txt

s	population	super_population	is_female	purple_hair	caffeine_consumption	six_toes
HG00096	GBR	EUR	false	false	5.0746e+01	false
HG00097	GBR	EUR	true	false	5.0244e+01	false
HG00098	GBR	EUR	false	false	6.3758e+01	false
HG00099	GBR	EUR	true	false	5.3899e+01	false
HG00100	GBR	EUR	true	false	4.1456e+01	false
HG00101	GBR	EUR	false	false	5.4906e+01	false
HG00102	GBR	EUR	true	false	3.8281e+01	false
HG00103	GBR	EUR	false	false	3.8200e+01	false
HG00104	GBR	EUR	true	false	5.1852e+01	false


We can import it as a [Hail Table](https://hail.is/docs/0.2/overview/table.html) with [hl.import_table](https://hail.is/docs/0.2/methods/impex.html?highlight=import_table#hail.methods.import_table).

We call it "sa" for "sample annotations".

In [17]:
sa = hl.import_table('resources/1kg_annotations.txt', 
                      impute=True, 
                      key='s')

2023-09-11 14:33:05.949 Hail: INFO: Reading table to impute column types
2023-09-11 14:33:07.189 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'population' as type str (imputed)
  Loading field 'super_population' as type str (imputed)
  Loading field 'is_female' as type bool (imputed)
  Loading field 'purple_hair' as type bool (imputed)
  Loading field 'caffeine_consumption' as type float64 (imputed)
  Loading field 'six_toes' as type bool (imputed)


While we can see the names and types of fields in the logging messages, we can also `show` this table:

In [18]:
sa.show()

,,,,,,
s,population,super_population,is_female,purple_hair,caffeine_consumption,six_toes
str,str,str,bool,bool,float64,bool
"""HG00096""","""GBR""","""EUR""",False,False,5.07e+01,False
"""HG00097""","""GBR""","""EUR""",True,False,5.02e+01,False
"""HG00098""","""GBR""","""EUR""",False,False,6.38e+01,False
"""HG00099""","""GBR""","""EUR""",True,False,5.39e+01,False
"""HG00100""","""GBR""","""EUR""",True,False,4.15e+01,False
"""HG00101""","""GBR""","""EUR""",False,False,5.49e+01,False
"""HG00102""","""GBR""","""EUR""",True,False,3.83e+01,False
"""HG00103""","""GBR""","""EUR""",False,False,3.82e+01,False


And we can `summarize` each field in `sa`:

In [19]:
sa.summarize()

2023-09-11 14:33:08.919 Hail: INFO: Coerced sorted dataset


Non-missing,3500 (100.00%)
Missing,0
Min Size,7
Max Size,7
Mean Size,7.00
Sample Values,"['HG00096', 'HG00097', 'HG00098', 'HG00099', 'HG00100']"
Non-missing,2819 (80.54%)
Missing,681 (19.46%)
Min Size,3
Max Size,3
Mean Size,3.00


## Add sample metadata into our 1KG `MatrixTable`

It just takes one line:

In [20]:
mt = mt.annotate_cols(pheno = sa[mt.s])

### What's going on here?

Understanding what's going on here is a bit more difficult. To understand, we need to understand a few pieces:

#### 1. `annotate` methods

In Hail, `annotate` methods refer to **adding new fields**. 

 - `MatrixTable`'s `annotate_cols` adds new column (**sample**) fields.
 - `MatrixTable`'s `annotate_rows` adds new row (**variant**) fields.
 - `MatrixTable`'s `annotate_entries` adds new entry (**genotype**) fields.
 - `Table`'s `annotate` adds new row fields.

In the above cell, we are adding a new column (**sample**) field called "pheno". This field should be the values in our table `sa` associated with the sample ID `s` in our `MatrixTable` - that is, this is performing a **join**.

Python uses square brackets to look up values in dictionaries:

    d = {'foo': 5, 'bar': 10}
    d['foo']

You should think of this in much the same way - for each column of `mt`, we are looking up the fields in `sa` using the sample ID `s`.

Let's look at where does this go into the `MatrixTable`

In [21]:
mt.describe(widget=True)

## Query the phenotype fields

What’s the fraction of samples with `purple_hair`?

In [22]:
mt.aggregate_cols(hl.agg.fraction(mt.pheno.purple_hair))

2023-09-11 14:33:11.710 Hail: WARN: aggregate_cols(): Aggregates over cols ordered by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


0.1457725947521866

How many people are in each self-reported major ancestry group?

In [23]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.super_population))

{'AFR': 76, 'AMR': 33, 'EAS': 75, 'EUR': 41, 'SAS': 62, None: 56}

## Sample QC

We'll start with examples of sample QC.

Hail has the function [hl.sample_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) to compute a list of useful statistics about samples from sequencing data. This function adds a new column field, `sample_qc`, with the computed statistics.

**Click the link** above to see the documentation, which lists the fields and their descriptions.

In [24]:
mt = hl.sample_qc(mt)

In [25]:
mt.describe(widget=True)

Let's visualize the distribution of `Mean DP` (`DP` = Read Depth) to `Call Rate`. First, we save the results of sample qc to a Hail Table file so that we can quickly access the results.

In [27]:
mt.cols().select('sample_qc').write(f'resources/sample_qc.ht', overwrite=True)
sample_qc = hl.read_table(f'resources/sample_qc.ht')

2023-09-11 14:33:35.620 Hail: INFO: Coerced sorted dataset
2023-09-11 14:33:36.109 Hail: INFO: wrote table with 343 rows in 10 partitions to resources/sample_qc.ht


In [28]:
from hail.ggplot import *

import plotly
import plotly.io as pio
pio.renderers.default='iframe'

fig = (
    ggplot(
        sample_qc,
        aes(
            x=sample_qc.sample_qc.dp_stats.mean, 
            y=sample_qc.sample_qc.call_rate, 
            tooltip=sample_qc.s,
            alpha=0.4
        )
    ) + 
    geom_point() +
    xlab("Mean DP") + 
    ylab("Call Rate") +
    ggtitle("DP by Call Rate")
)
fig.show()

### Filter columns using generated QC statistics

Before filtering samples, we should compute a raw sample count:

In [ ]:
mt.count_cols()

`filter_cols` removes entire columns from the matrix table. Here, we keep columns (samples) where the `call_rate` is over 95%:

In [ ]:
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.95)


We can compute a final sample count:

In [ ]:
mt.count_cols()

Let's inspect the distribution of sequencing depth across all genotypes.

In [ ]:
(
    ggplot(
        mt,
        aes(
            x=mt.DP,
        )
    ) + 
    geom_histogram(min_val=0, max_val=30, bins=30) +
    ggtitle("DP")
)

## Variant QC

Hail has the function [hl.variant_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) to compute a list of useful statistics about **variants** from sequencing data.

Once again, **Click the link** above to see the documentation!

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
mt.describe(widget=True)

We can `show()` the computed information:

In [ ]:
mt.variant_qc.show()

Metrics like `call_rate` are important for QC. Let's plot the cumulative density function of call rate per variant:

In [ ]:
mt.filter_rows(hl.agg.call_stats(mt.GT, mt.alleles).AF[1] > 0.01).show()

In [ ]:
(
    ggplot(
        mt,
        aes(x=mt.variant_qc.call_rate)
    ) + 
    geom_density() +
    ggtitle("Call Rate approximate PDF")
)

Before filtering variants, we should compute a raw variant count:

In [ ]:
# pre-qc variant count
mt.count_rows()

`filter_rows` removes entire rows of the matrix table. Here, we keep rows where the `call_rate` is over 95%:

In [ ]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95)

After filtering, we can see more resolution of the top end of the call rate distribution:

In [ ]:
(
    ggplot(
        mt,
        aes(x=mt.variant_qc.call_rate)
    ) + 
    geom_density() +
    ggtitle("Call Rate approximate PDF")
)

We can then compute the final sample and variant count:

In [ ]:
mt.count()

# Association Testing and PCA

We will first filter to common variants (those with an alternate allele frequency over 1%). GWAS cannot detect signal from extremely rare variants, like those only observed in one individual.

In [ ]:
mt = mt.filter_rows(hl.agg.call_stats(mt.GT, mt.alleles).AF[1] > 0.01)

In [ ]:
mt.describe(widget=True)

Remember that in a GWAS we're running independent association tests on each variant.

In Hail, the method we use is [hl.linear_regression_rows](https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows). Why isn't this called `hl.gwas`? Modularity!

We use the phenotype `caffeine_consumption` as our dependent variable, the number of alternate alleles as our independent variable, and no covariates besides an intercept term (that's the `1.0`).

In [ ]:
gwas = hl.linear_regression_rows(y=mt.pheno.caffeine_consumption, 
                                 x=mt.GT.n_alt_alleles(), 
                                 covariates=[1.0])

In [ ]:
gwas.describe(widget=True)

## Visualization

Let’s visualize our association test results from the linear regression. We can do so by creating 2 common plots: a [Manhattan plot](https://en.wikipedia.org/wiki/Manhattan_plot) and a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot).

We'll start with the Manhattan plot:

In [ ]:
gwas.write(f'results/gwas.ht')
gwas = hl.read_table(f'results/gwas.ht')

In [ ]:
tooltip = hl.str(gwas.locus) + "; " + hl.str(gwas.p_value)

(
    ggplot(gwas, aes(x=gwas.locus, tooltip=tooltip)) + 
    geom_point(aes(y=-hl.log10(gwas.p_value),
                   color=gwas.locus.contig_idx%2)) + 
    geom_hline(yintercept=7.3, color='red')
)

The other common plot is the Q-Q (quantile-quantile) plot.

In [ ]:
n_tests = gwas.count()

ht = gwas.key_by(gwas.p_value).add_index()

(
    ggplot(
        ht,
        aes(
            x=-hl.log10(ht.idx/n_tests),
        )
    ) + 
    geom_point(aes(y=-hl.log10(ht.p_value))) +
    geom_line(aes(y=-hl.log10(ht.idx/n_tests)),
              color="red") +
    xlab("Expected") + 
    ylab("Observed") +
    ggtitle("QQ Plot")
)

## Confounded!

The Q-Q plot indicates **extreme** inflation of p-values.

What do you think is a good range for genomic inflation?

If you've done a GWAS before, you've probably included a few other covariates that might be confounders -- age, sex, and principal components.

Principal components are a measure of genetic ancestry, and can be used to control for [population stratification](https://en.wikipedia.org/wiki/Population_stratification).

Principal component analysis (PCA) is a very general statistical method for reducing high dimensional data to a small number of dimensions which capture most of the variation in the data. Hail has the function [pca](https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca) for performing generic PCA.

PCA typically works best on normalized data (e.g. mean centered). Hail provides the specialized function [hwe_normalized_pca](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca) which first normalizes the genotypes according to the Hardy-Weinberg Equilibium model.

In [ ]:
pca_eigenvalues, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, compute_loadings=True)

The pca function returns three things:
* **eigenvalues**: an array of doubles
* **scores**: a table keyed by sample
* **loadings**: a table keyed by variant

The **loadings** are the *principal directions*, each of which is a weighted sum of variants (a "virtual variant"). By default, `hwe_normalized_pca` gives us 10 principal directions.

In [ ]:
mt = mt.annotate_cols(pca = pca_scores[mt.s])

## Control confounders and run another GWAS

Now that we have computed principal components and saved it into our `mt`, let’s attempt to adjust the inflation that we saw in our initial Q-Q plot e.g.  `mt.pheno.is_female`. We will now add PCs as `covariates` in `linear_regression_rows`:


In [ ]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.caffeine_consumption, 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pca.scores[0], mt.pca.scores[1], mt.pca.scores[2]])

In [ ]:
n_tests = gwas.count()

ht = gwas.key_by(gwas.p_value).add_index()

(
    ggplot(
        ht,
        aes(
            x=-hl.log10(ht.idx/n_tests),
        )
    ) + 
    geom_point(aes(y=-hl.log10(ht.p_value))) +
    geom_line(aes(y=-hl.log10(ht.idx/n_tests)),
              color="red") +
    xlab("Expected") + 
    ylab("Observed") +
    ggtitle("QQ Plot")
)

The above Q-Q plot indicates much better (but not good!) genomic control. Let's try the Manhattan plot:

In [ ]:
tooltip = hl.str(gwas.locus) + "; " + hl.str(gwas.p_value)

(
    ggplot(gwas, aes(x=gwas.locus, tooltip=tooltip)) + 
    geom_point(aes(y=-hl.log10(gwas.p_value),
                   color=gwas.locus.contig_idx%2)) + 
    geom_hline(yintercept=7.3, color='red')
)

### What other covariates can you think off that could possibly clean up this analysis?

#### Zoom Breakout rooms Activity

We will assign you into TWO breakout rooms. 

**Team/Room _Purple Hair_**

Create a model with **purple hair** as the outcome


**Team/Room _Polydactylism_**

Create a model with **six toes** as the outcome

Your assignment would be to :

1) What is the distribution of people who have the phenotype? A simple list with do from `count()` or `show()`! 

2) Create a logistic model with the given phenotype outcome using [Hail documentation](https://hail.is/docs/0.2/methods/stats.html#hail.methods.logistic_regression_rows). Use the search function at the top of the documentation page if you need more information!  

3) Print QQ plot

4) Print Manhattan plot

# Write QC'ed final dataset to disk

In [ ]:
mt.write('output/post_qc.mt', overwrite=True)